In [1]:
import numpy as np
import torch

import sys
sys.path.insert(0, '../ml')
from customlayers import *

In [2]:
2*2*2*2

16

In [3]:
hparams = {'accelerator': 'gpu',
           'devices': [0],
           'dtype': torch.float64,
           'precision': 64,
           'loss_terms': [F.mse_loss],
           'loss_coeffs': [1],
           'optimizer': torch.optim.Adam, 
           'learning_rate': 1e-4,
           'batch_size': 100,
           'max_epochs': 5000,
           'early_stopping_patience': 1000000,
           'DeepONet': False,
           'VarMiON': True,
           'NGO': False,
           'Q': 12,
           'Q_L': 268,
           'h': 64, 
           'Petrov-Galerkin': False,
           'bias_NLBranch': False,
           'bias_LBranch': False,
           'NLB_outputactivation': nn.Tanhshrink(),
           'Cholesky': False,
           'scale_invariance': False,
           'norm_basis': True,
           'bound_mus': False,
           '1/theta': False,
           'symgroupavg': False,
           'NOMAD': False}

params = {}
params['hparams'] = {}
params['hparams'] = hparams

In [4]:
class UNet(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.hparams = params['hparams']
        self.layers = nn.ModuleList()
        self.num_channels = 30
        # Adjusted convolutional layers
        self.layers.append(ReshapeLayer(output_shape=(1,self.hparams['h'],self.hparams['h'])))
        self.layers.append(nn.Conv2d(in_channels=1, out_channels=self.num_channels, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv2d(in_channels=self.num_channels, out_channels=2*self.num_channels, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv2d(in_channels=2*self.num_channels, out_channels=1*self.num_channels, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv2d(in_channels=1*self.num_channels, out_channels=1, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer(output_shape=(int(1*(self.hparams['h']/16)**2),)))
        self.layers.append(nn.Linear(in_features=int(1*(self.hparams['h']/16)**2),out_features=int(1*(self.hparams['h']/16)**2)))
        self.layers.append(nn.ReLU())
        self.layers.append(ReshapeLayer(output_shape=(1,int(self.hparams['h']/16),int(self.hparams['h']/16))))
        self.layers.append(nn.ConvTranspose2d(1, self.num_channels, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.ConvTranspose2d(self.num_channels, 2*self.num_channels, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.ConvTranspose2d(2*self.num_channels, 1*self.num_channels, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.ConvTranspose2d(1*self.num_channels, 1, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer(output_shape=(self.hparams['h'],self.hparams['h'])))
        if self.hparams['NLB_outputactivation'] is not None:
            self.layers.append(self.hparams['NLB_outputactivation'])

    def forward(self, x):
        x0 = x
        x1 = self.layers[0](x0)
        x2 = self.layers[1](x1)
        x3 = self.layers[2](x2)
        x4 = self.layers[3](x3)
        x5 = self.layers[4](x4)
        x6 = self.layers[5](x5)
        x7 = self.layers[6](x6)
        x8 = self.layers[7](x7)
        x9 = self.layers[8](x8)
        x10 = self.layers[9](x9)
        x11 = self.layers[10](x10) + x9
        x12 = self.layers[11](x11)
        x13 = self.layers[12](x12)
        x14 = self.layers[13](x13) + x7
        x15 = self.layers[14](x14)
        x16 = self.layers[15](x15) + x5
        x17 = self.layers[16](x16)
        x18 = self.layers[17](x17) + x3
        x19 = self.layers[18](x18) + x1
        x20 = self.layers[19](x19)
        x21 = self.layers[20](x20)
        y = x21
        return y

In [5]:
x = torch.rand(hparams['batch_size'],hparams['h'],hparams['h'])

In [6]:
model = UNet(params)

In [7]:
sum(p.numel() for p in model.parameters())

29552

In [8]:
model.forward(x)

tensor([[[ 1.2804e-02,  3.8973e-03,  1.3876e-01,  ...,  3.9982e-02,
           3.7415e-01,  1.9780e-01],
         [ 4.0778e-02,  8.2344e-03,  9.5532e-03,  ...,  2.1135e-01,
           1.0635e-01,  4.5960e-04],
         [ 3.9281e-02,  8.3237e-01,  8.4772e-02,  ...,  1.6979e-01,
           1.7192e-04,  3.6278e-01],
         ...,
         [ 9.1115e-02,  5.9623e-06,  9.8518e-03,  ...,  2.0320e-01,
           1.0824e-01,  1.1338e-01],
         [ 9.1620e-03,  3.3493e-01,  1.3121e-01,  ...,  4.9063e-01,
           5.4177e-05,  2.9967e-02],
         [ 5.2140e-02,  1.3117e-02,  9.5292e-02,  ..., -4.8058e-04,
           7.7246e-03,  5.8889e-03]],

        [[ 2.3735e-03,  3.4624e-01,  3.9858e-03,  ...,  4.5389e-01,
           2.4706e-02,  3.7337e-02],
         [ 4.7612e-04, -2.8990e-05,  6.5114e-02,  ...,  1.2342e-01,
           5.3806e-03, -2.8033e-07],
         [ 3.0117e-01,  6.3709e-01,  2.8061e-01,  ...,  3.2392e-01,
           1.7693e-01,  3.9921e-01],
         ...,
         [ 1.2515e-03,  5

In [9]:
4*4*3*3

144

In [10]:
class UNet(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.hparams = params['hparams']
        self.layers = nn.ModuleList()
        self.num_channels = 30
        self.kernel_sizes = [4,4,3,3]
        # Adjusted convolutional layers
        self.layers.append(ReshapeLayer(output_shape=(1,self.hparams['h'],self.hparams['h'])))
        self.layers.append(nn.Conv2d(in_channels=1, out_channels=self.num_channels, kernel_size=4, stride=4, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=self.num_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.Conv2d(in_channels=self.num_channels, out_channels=2*self.num_channels, kernel_size=4, stride=4, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=2*self.num_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.Conv2d(in_channels=2*self.num_channels, out_channels=self.num_channels, kernel_size=3, stride=3, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv2d(in_channels=self.num_channels, out_channels=1, kernel_size=3, stride=3, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer(output_shape=(int(1*(self.hparams['h']/144)**2),)))
        self.layers.append(nn.Linear(in_features=int(1*(self.hparams['h']/144)**2),out_features=int(1*(self.hparams['h']/144)**2)))
        self.layers.append(nn.ReLU())
        self.layers.append(ReshapeLayer(output_shape=(1,int(self.hparams['h']/144),int(self.hparams['h']/144))))
        self.layers.append(nn.ConvTranspose2d(1, self.num_channels, kernel_size=3, stride=3, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=self.num_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.ConvTranspose2d(self.num_channels, 2*self.num_channels, kernel_size=3, stride=3, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=2*self.num_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.ConvTranspose2d(2*self.num_channels, self.num_channels, kernel_size=4, stride=4, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.ConvTranspose2d(self.num_channels, 1, kernel_size=4, stride=4, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer(output_shape=(self.hparams['h'],self.hparams['h'])))
        if self.hparams['NLB_outputactivation'] is not None:
            self.layers.append(self.hparams['NLB_outputactivation'])

            
    def forward(self, x):
        x0 = x
        x1 = self.layers[0](x0)
        x2 = self.layers[1](x1)
        x3 = self.layers[2](x2)
        x4 = self.layers[3](x3)
        x5 = self.layers[4](x4)
        x6 = self.layers[5](x5)
        x7 = self.layers[6](x6)
        x8 = self.layers[7](x7)
        x9 = self.layers[8](x8)
        x10 = self.layers[9](x9) 
        x11 = self.layers[10](x10)
        x12 = self.layers[11](x11)
        x13 = self.layers[12](x12) + x11
        x14 = self.layers[13](x13)
        x15 = self.layers[14](x14)
        x16 = self.layers[15](x15) + x9
        x17 = self.layers[16](x16)
        x18 = self.layers[17](x17)
        x19 = self.layers[18](x18) + x6
        x20 = self.layers[19](x19)
        x21 = self.layers[20](x20) 
        x22 = self.layers[21](x21) + x3
        x23 = self.layers[22](x22) + x1
        x24 = self.layers[23](x23)
        x25 = self.layers[24](x24)
        y = x25
        return y

In [11]:
x = torch.rand(hparams['batch_size'],hparams['h'],hparams['h'])

In [12]:
model = UNet(params)

/home/prins/st8/anaconda3/lib/python3.10/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [13]:
sum(p.numel() for p in model.parameters())

91860

In [14]:
model.forward(x)

RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size

In [27]:
class UNet(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.hparams = params['hparams']
        self.layers = nn.ModuleList()
        self.num_channels = 24
        self.kernel_sizes = [4,4,2,2]
        # Adjusted convolutional layers
        self.layers.append(ReshapeLayer(output_shape=(1,self.hparams['h'],self.hparams['h'])))
        self.layers.append(nn.Conv2d(in_channels=1, out_channels=self.num_channels, kernel_size=self.kernel_sizes[0], stride=self.kernel_sizes[0], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=self.num_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.Conv2d(in_channels=self.num_channels, out_channels=self.num_channels, kernel_size=self.kernel_sizes[1], stride=self.kernel_sizes[1], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=self.num_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.Conv2d(in_channels=self.num_channels, out_channels=self.num_channels, kernel_size=self.kernel_sizes[2], stride=self.kernel_sizes[2], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv2d(in_channels=self.num_channels, out_channels=self.num_channels, kernel_size=self.kernel_sizes[3], stride=self.kernel_sizes[3], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer(output_shape=(int(self.num_channels*(self.hparams['h']/144)**2),)))
        self.layers.append(nn.Linear(in_features=int(self.num_channels*(self.hparams['h']/144)**2),out_features=int(self.num_channels*(self.hparams['h']/144)**2)))
        self.layers.append(nn.ReLU())
        self.layers.append(ReshapeLayer(output_shape=(self.num_channels,int(self.hparams['h']/144),int(self.hparams['h']/144))))
        self.layers.append(nn.ConvTranspose2d(self.num_channels, self.num_channels, kernel_size=self.kernel_sizes[3], stride=self.kernel_sizes[3], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=self.num_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.ConvTranspose2d(self.num_channels, self.num_channels, kernel_size=self.kernel_sizes[2], stride=self.kernel_sizes[2], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=self.num_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.ConvTranspose2d(self.num_channels, self.num_channels, kernel_size=self.kernel_sizes[1], stride=self.kernel_sizes[1], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.ConvTranspose2d(self.num_channels, 1, kernel_size=self.kernel_sizes[0], stride=self.kernel_sizes[0], bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer(output_shape=(self.hparams['h'],self.hparams['h'])))
        if self.hparams['NLB_outputactivation'] is not None:
            self.layers.append(self.hparams['NLB_outputactivation'])

            
    def forward(self, x):
        x0 = x
        x1 = self.layers[0](x0)
        x2 = self.layers[1](x1)
        x3 = self.layers[2](x2)
        x4 = self.layers[3](x3)
        x5 = self.layers[4](x4)
        x6 = self.layers[5](x5)
        x7 = self.layers[6](x6)
        x8 = self.layers[7](x7)
        x9 = self.layers[8](x8)
        x10 = self.layers[9](x9) 
        x11 = self.layers[10](x10)
        x12 = self.layers[11](x11)
        x13 = self.layers[12](x12) + x11
        x14 = self.layers[13](x13)
        x15 = self.layers[14](x14)
        x16 = self.layers[15](x15) + x9
        x17 = self.layers[16](x16)
        x18 = self.layers[17](x17)
        x19 = self.layers[18](x18) + x6
        x20 = self.layers[19](x19)
        x21 = self.layers[20](x20) 
        x22 = self.layers[21](x21) + x3
        x23 = self.layers[22](x22) + x1
        x24 = self.layers[23](x23)
        x25 = self.layers[24](x24)
        y = x25
        return y

In [28]:
x = torch.rand(hparams['batch_size'],hparams['h'],hparams['h'])

In [29]:
model = UNet(params)

In [30]:
sum(p.numel() for p in model.parameters())

28628

In [16]:
model.forward(x)

RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size